In [9]:
import requests
import json
import datetime

In [2]:
import predict_imb_price

SI_FC, last_si, quantiles = predict_imb_price.pred_SI(dev='cpu')

print(last_si)


C:\Users\u0138303\OneDrive - KU Leuven\StorageOptimisationImbalance\Scripts\decorators.py:17: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  dates = pd.date_range(start, end, freq=freq, closed="left")  # Python 3.9 or below
C:\Users\u0138303\OneDrive - KU Leuven\StorageOptimisationImbalance\Scripts\Data_Elia_API.py:354: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return_df.loc[len(df_qh)] = [latest_qh,est_SI_current]
C:\Users\u0138303\OneDrive - KU Leuven\StorageOptimisationImbalance\Scripts\Data_Elia_API.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'systemimbalan

realtime
mostrecentforecast
dayahead11hforecast
realtime
mostrecentforecast
dayahead11hforecast
realtime
mostrecentforecast
dayahead11hforecast
realtime
mostrecentforecast
dayahead11hforecast


C:\Users\u0138303\OneDrive - KU Leuven\StorageOptimisationImbalance\Scripts\Data_Elia_API.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['realtime'] = np.maximum(df['realtime'].mask(df['realtime'].isna(),df['mostrecentforecast']+rt_vs_mrf),0)


realtime
mostrecentforecast
dayahead11hforecast
realtime
mostrecentforecast
dayahead11hforecast
realtime
mostrecentforecast
dayahead11hforecast
realtime
mostrecentforecast
dayahead11hforecast


C:\Users\u0138303\OneDrive - KU Leuven\StorageOptimisationImbalance\Scripts\decorators.py:17: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  dates = pd.date_range(start, end, freq=freq, closed="left")  # Python 3.9 or below
C:\Users\u0138303\OneDrive - KU Leuven\StorageOptimisationImbalance\Scripts\Data_Elia_API.py:314: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['measured'] = df['measured'].mask(df['measured'].isna(),df['mostrecentforecast']*rt_vs_mrf_rel)
C:\Users\u0138303\OneDrive - KU Leuven\StorageOptimisationImbalance\Scripts\decorators.py:17: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  dates = pd.date_range(start, end, freq=freq, closed="left")  # Python 3.9 or below


realtime
mostrecentforecast
dayahead11hforecast
realtime
mostrecentforecast
dayahead11hforecast
realtime
mostrecentforecast
dayahead11hforecast
realtime
mostrecentforecast
dayahead11hforecast
realtime
mostrecentforecast
dayahead11hforecast
realtime
mostrecentforecast
dayahead11hforecast
realtime
mostrecentforecast
dayahead11hforecast
realtime
mostrecentforecast
dayahead11hforecast
realtime
mostrecentforecast
dayahead11hforecast
realtime
mostrecentforecast
dayahead11hforecast


C:\Users\u0138303\OneDrive - KU Leuven\StorageOptimisationImbalance\Scripts\decorators.py:17: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  dates = pd.date_range(start, end, freq=freq, closed="left")  # Python 3.9 or below


Total time for loading data: 11.444528818130493s
(-7.3380833333333335, Timestamp('2023-12-02 00:00:00+0000', tz='UTC'))


In [3]:
len(quantiles)

9

In [6]:
SI_FC[1,:],last_si[0]

(array([-431.18643, -334.8008 , -284.54404, -202.64388, -114.62644,
         -32.22912,   37.55415,   80.36451,  165.76802], dtype=float32),
 -7.3380833333333335)

In [11]:
id = 10
api_link = "https://swdd9r1vei.execute-api.eu-north-1.amazonaws.com/items"

prices_fc_spread = dict()

prices_fc_spread = dict()
last_si_time = last_si[1]
fc_times = [(last_si_time + datetime.timedelta(minutes=15 * fc_step)).strftime("%H:%M:%S") for fc_step in range(8)]

for i, fc_time in enumerate(fc_times):
    prices_fc_spread[fc_time] = [str(SI_FC[i,q]) for q in range(SI_FC.shape[1]) ]


data = {
    "id": id,
    "time": last_si_time.strftime('%Y-%m-%d %H:%M:%S'),
    "imba_price": last_si[0],
    "imba_price_fc": 1,
    "charge": 1,
    "soc": 1,
    "fc_spread": prices_fc_spread,
    "quantiles": quantiles
}

# data = {
#     "id": id * j,
#     "time": some_time_str,
#     "imba_price": imba_price,
#     "imba_price_fc": imba_price_fc,
#     "charge": charge,
#     "soc": soc,
#     "fc_spread": prices_fc_spread
# }

print(f"Writing now to API:",last_si_time)

response = requests.put(api_link, json=data)
print(response.text)

Writing now to API: 2023-12-02 00:00:00+00:00
"Put item 10"


'19:30:00'

In [39]:
import datetime
import pandas as pd
import os
import requests
import numpy as np

api_link = "https://swdd9r1vei.execute-api.eu-north-1.amazonaws.com/items"
ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
prices = [0, 100, 50, 40, 200, 100, 20, 0, 100, 100]
prices_fc = [10, 90, 100, 30, 180, 200, 80, 0, 40, 0]

median_fc = np.median(prices_fc)
charges = [1 if fc < median_fc else -1 for fc in prices_fc]

socs = [5]
soc = 5

time = datetime.datetime.now()
time_str = time.strftime("%H:%M:%S")

for charge in charges:
    soc = soc + charge
    socs.append(soc)


for j in range(1, 2, 1):
    for id, imba_price, imba_price_fc, charge, soc in zip(ids, prices, prices_fc, charges, socs):
        some_time = time + datetime.timedelta(minutes=15 * id)
        some_time_str = some_time.strftime("%H:%M:%S")
        prices_fc_spread = dict()
        fc_times = [(some_time + datetime.timedelta(minutes=15 * fc_step)).strftime("%H:%M:%S") for fc_step in range(8)]

        for i, fc_time in enumerate(fc_times):
            prices_fc_spread[fc_time] = [imba_price * 0.9 ** i, imba_price * 1.1 ** i]

        data = {
            "id": id * j,
            "time": some_time_str,
            "imba_price": imba_price,
            "imba_price_fc": imba_price_fc,
            "charge": charge,
            "soc": soc,
            "fc_spread": prices_fc_spread
        }

        # print(f"Testing API get", current_time)
        # #
        # # response = requests.get("https://swdd9r1vei.execute-api.eu-north-1.amazonaws.com/items")
        # # print(response.text)
        #
        if id =="1":
            print(f"Writing now to API:", some_time_str)
    
            response = requests.put(api_link, json=data)
            print(response.text)